In [1]:
import os

import cv2
%load_ext autoreload
%autoreload 2

In [2]:
import tqdm
import glob

trajs = []

for i in range(4):
    out = sorted(glob.glob(f'sols/sol{i}/traj*_*.pkl'))
    for idx, j in enumerate(tqdm.tqdm(out, total=len(out))):
        sol_path = j
        goal_path = ('_'.join(j.split('_')[:-1])+'.pkl').replace('sols/sol', 'data/')
        trajs.append([sol_path, goal_path])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 210/210 [00:00<00:00, 831731.67it/s]


In [3]:
for i in range(4):
    out = sorted(glob.glob(f'obey/obey{i}/traj*_*.pkl'))
    for idx, j in enumerate(tqdm.tqdm(out, total=len(out))):
        sol_path = j
        goal_path = ('_'.join(j.split('_')[:-1])+'.pkl').replace('obey/obey', 'data/')
        trajs.append([sol_path, goal_path])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 350/350 [00:00<00:00, 1031627.83it/s]


In [4]:
for i in range(4):
    out = sorted(glob.glob(f'sol_remain{i}/traj*_*.pkl'))
    for idx, j in enumerate(tqdm.tqdm(out, total=len(out))):
        sol_path = j
        goal_path = ('_'.join(j.split('_')[:-1])+'.pkl').replace('sol_remain', 'data/')
        trajs.append([sol_path, goal_path])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140/140 [00:00<00:00, 948630.95it/s]


In [5]:
"""
import os
import lzma
os.makedirs('cached_state_path/init', exist_ok=True)
os.makedirs('cached_state_path/target', exist_ok=True)
import torch
import pickle
import numpy as np
print(len(trajs))
for idx, (sol, goal) in enumerate(tqdm.tqdm(trajs, total=len(trajs))):
    a = torch.load(sol)
    b = torch.load(goal)

    initial_state = a['initial_state']
    goal_state = b[1]

    #out = execute(env, a['initial_state'], a['best_action'], filename='array', img_size=64)
    #print(initial_state.keys(), goal_state.keys())
    init = initial_state['state'][0]
    with lzma.open(f'cached_state_path/init/state_{idx}.xz', 'wb') as f:
        pickle.dump(initial_state, f)

    goal = goal_state['state'][0]
    np.save(f'cached_state_path/target/target_{idx}.npy', goal)
    """

"\nimport os\nimport lzma\nos.makedirs('cached_state_path/init', exist_ok=True)\nos.makedirs('cached_state_path/target', exist_ok=True)\nimport torch\nimport pickle\nimport numpy as np\nprint(len(trajs))\nfor idx, (sol, goal) in enumerate(tqdm.tqdm(trajs, total=len(trajs))):\n    a = torch.load(sol)\n    b = torch.load(goal)\n\n    initial_state = a['initial_state']\n    goal_state = b[1]\n\n    #out = execute(env, a['initial_state'], a['best_action'], filename='array', img_size=64)\n    #print(initial_state.keys(), goal_state.keys())\n    init = initial_state['state'][0]\n    with lzma.open(f'cached_state_path/init/state_{idx}.xz', 'wb') as f:\n        pickle.dump(initial_state, f)\n\n    goal = goal_state['state'][0]\n    np.save(f'cached_state_path/target/target_{idx}.npy', goal)\n    "

In [6]:
from plb.cut.sample_utils import init_env, execute
env, state = init_env()

[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-1pugjf11
[Taichi] version 0.7.26, llvm 10.0.0, commit e37bdb5e, linux, python 3.7.11
[I 09/30/21 14:32:27.329 2938380] [shell.py:_shell_pop_print@35] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=cuda
[Tina] version 0.1.1
[Tina] Taichi properties hacked
pimirives: num primitive: 2
Building primitive
action:
  dim: 3
  scale: (0.015, 0.015, 0.015)
collision_group: [0.0, 0.0, 0.0]
color: (0.7568, 0.6039, 0.4196)
friction: 0.0
h: (0.15, 0.15)
init_pos: (0.5, 0.3, 0.5)
init_rot: (1.0, 0.0, 0.0, 0.0)
lower_bound: (0.0, 0.0, 0.0)
prot: (1.0, 0.0, 0.0, 0.58)
shape: Knife
size: (0.025, 0.2, 0.06)
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 0
  scale: ()
collision_group: [0.0, 0.0, 0.0]
color: (0.3, 0.3, 0.3)
friction: 0.9
h: (0.15, 0.15)
init_pos: (0.3, 0.3, 0.3)
init_rot: (1.0, 0.0, 0.0, 0.0)
lower_bound: (0.0, 0.0, 0.0)
prot: (1.0, 0.0, 0.0, 0.58)
shape: 


In [7]:
from imitation.env_spec import set_render_mode
class ENV:
    def __init__(self):
        self.taichi_env = env
set_render_mode(ENV(), 'CutRearrange-v1')


In [8]:
env.renderer.update_camera((0.52, 0.3, 0.6), 0., -1.3, 0.53)
img = env.render('rgb', img_size=512)
plt.imshow(img[:, :,:3])
plt.show()

NameError: name 'plt' is not defined

In [ ]:
import torch
import numpy as np
target_imgs = []
for idx, (sol, goal) in enumerate(tqdm.tqdm(trajs, total=len(trajs))):
    goal_state = torch.load(goal)
    env.set_state(**goal_state[1])
    img = env.render('rgb', spp=1, img_size=64)

    target_imgs.append(img)
    cv2.imwrite(f'cached_state_path/target/target_{idx}.png', cv2.cvtColor(np.uint8(img[:,:,:3] * 255), cv2.COLOR_RGB2BGR))

np.save('cached_state_path/target/target_imgs.npy', target_imgs)
